In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
#đọc file training
training_df = pd.read_excel(r"/content/gdrive/MyDrive/KLTN/20210421/Data/training.xlsx")
training_df = training_df.rename(columns={'Column1':'User', 'Column2':'Item', 'Column3':'Rating'}, errors="raise")
del training_df["Column4"]

In [ ]:
#đọc file testing
testing_df = pd.read_excel(r"/content/gdrive/MyDrive/KLTN/20210421/Data/testing.xlsx")
testing_df = testing_df.rename(columns={'Column1':'User', 'Column2':'Item', 'Column3':'Rating'}, errors="raise")
del testing_df["Column4"]

In [ ]:
#Hàm trả về danh sách item tư vấn cho người dùng
#Input: 
#user - id người dùng đang xét, N - số lượng item, training_df - file training, sim_path - đường dẫn thư mục độ tương đồng
#Output:
#Danh sách N item tư vấn cho người dùng
def recommend_list(user, N, training_df, sim_path):

  item_col=[]
  rating_col=[]

  #Lấy ra độ tương đồng của người dùng 
  user_df = pd.read_excel(r"{0}/{1}.xlsx".format(sim_path, user))

  #các item của người dùng có đánh giá tính cực
  user_item = training_df.Item[training_df.User == user][training_df.Rating == 1].to_list()

  #danh sách các item cần dự đoán
  predict_list = [value for value in range(1,3953) if value not in user_item]

  #vòng lặp item
  for j in predict_list:
    #Danh sách những người dùng có đánh giá item j  
    user_j = training_df.User[training_df.Item == j].to_list()

    #Tổng sim 10 người dùng (l = 10) có đánh giá tới item j và có độ tương đồng lớn nhất
    sum_sim = sum(user_df.Sim[user_df.User.isin(user_j) == True][:10].to_list())

    #thêm id item j vào cột item
    item_col.append(j)

    #Thêm sim vào cột đánh giá
    rating_col.append(sum_sim)
  
   #Tạo dataframe từ cột item và cột đánh giá
  df = pd.DataFrame(data={'Item':item_col,'Rating':rating_col})

  #Sắp xếp đánh giá dự đoán theo giá trị đánh giá
  df = df.sort_values(by=['Rating'],axis=0,ascending=False, ignore_index=True)

  #Lấy danh sách item từ 1 tới N
  df =  df.iloc[:N]

  #Trả về danh sách N item 
  return df.Item.to_list()

In [ ]:
#Hàm đánh giá kết quả dự đoán
#Input: 
#sim_path: đường dẫn file tương đồng, save_path: nơi lưu trữ, testing_df - file test, training_df - file training
#first - id người dùng đầu tiên trong vòng lặp, last - id người dùng cuối cùng trong vòng lặp
#Output:
#File excel chứa kết quả đánh giá thuật toán
def revaluation(N, sim_path, save_path, testing_df, training_df, first, last):

  #Khai bao trước những thước đo đánh giá thành các cột
  User_col = []
  Prec_u_N = []
  Rec_u_N = []
  F1_u_N = [] 

  #Vòng lặp người dùng với first -> last
  for i in range(first, last + 1):
    #Danh sách item system recommend cho người dùng
    I_N = recommend_list(i, N, training_df, sim_path)
    #Danh sách item trong file test của người dùng
    I_te = testing_df.Item[testing_df.User == i][testing_df.Rating == 1].to_list()
    #Phần giao giữa danh sách item 
    intersection = [value for value in I_N if value in I_te]
    try:
      #Tính thước đo đánh giá
      Prec =  abs(len(intersection)) / N
      Rec = abs(len(intersection)) / abs(len(I_te))
      F1 = 2 * (Prec * Rec) / (Prec + Rec)
    except:
      #Trường hợp các list này rỗng 
      if intersection == [] or I_te == [] or I_N == []:
        Prec = 0
        Rec = 0
        F1 = 0

    #Thêm id người dùng, các độ đo vào các cột
    User_col.append(i)
    Prec_u_N.append(Prec)
    Rec_u_N.append(Rec)
    F1_u_N.append(F1)  
    
    print("User:", i," ", "Prec:", Prec," ", "Rec:", Rec, " ", "F1:", F1)

  #Tạo dataframe 
  df = pd.DataFrame(data={'User':User_col, 'Precision':Prec_u_N, 'Recursion': Rec_u_N, 'F1': F1_u_N})

  #Xuất file
  df.to_excel(save_path, index = False)

In [ ]:
revaluation(10,"/content/gdrive/MyDrive/KLTN/20210421/K-NN/Jaccard Sim", "/content/gdrive/MyDrive/KLTN/20210421/K-NN/K-NN revaluation/revaluation.xlsx",testing_df, training_df, 3000, 3001)